In [21]:
import pymongo
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [22]:
!which chromedriver

which: no chromedriver in (/c/Users/dshakoory/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/dshakoory/bin:/c/Program Files (x86)/Common Files/Oracle/Java/javapath:/c/WINDOWS/system32:/c/WINDOWS:/c/WINDOWS/System32/Wbem:/c/WINDOWS/System32/WindowsPowerShell/v1.0:/c/Program Files (x86)/IBM/Client Access/Emulator:/c/Program Files (x86)/IBM/Client Access/Shared:/c/Program Files (x86)/IBM/Client Access:/c/Program Files (x86)/Shoreline Communications/ShoreWare Client:/c/Program Files (x86)/Shoreline Communications/ShoreWare Client/win64:/c/Program Files (x86)/Enterprise Vault/EVClient:/c/Program Files (x86)/Microsoft SQL Server/Client SDK/ODBC/130/Tools/Binn:/c/Program Files (x86)/Microsoft SQL Server/140/Tools/Binn:/c/Program Files (x86)/Microsoft SQL Server/140/DTS/Binn:/c/Program Files (x86)/Microsoft SQL Server/140/Tools/Binn/ManagementStudio:/c/Program Files (x86)/Symantec/VIP Access Client:/c/WINDOWS/System32/OpenSSH:/c/Program Files (x86)/Intel/Intel

In [23]:
#setting executive path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [24]:
#getting to Nasa news site
news_url ="https://mars.nasa.gov/news/"
browser.visit(news_url)

In [5]:
news_html = browser.html
nsoup = bs(news_html,'lxml')

In [6]:
news_title = nsoup.find('div', class_='content_title').text
news_p = nsoup.find('div', class_='article_teaser_body').text

### JPL Mars Space Images - Featured Image

In [7]:
#visiting the URL
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

In [8]:
#finding the image and clicking
jpl_html = browser.html
jsoup = bs(jpl_html,'lxml')

In [9]:
img_link = jsoup.find('div',class_='carousel_container').article.footer.a['data-fancybox-href']
base_link = jsoup.find('div', class_='jpl_logo').a['href'].rstrip('/')
featured_image_url = base_link + img_link
featured_image_title = jsoup.find('h1', class_="media_feature_title").text.strip()

### Mars Weather

In [10]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [11]:
w_html = browser.html
wsoup = bs(w_html,'lxml')

In [12]:
#searching through the tweet for the text of the tweets that contains p tag
tweets = wsoup.find_all('p',class_='TweetTextSize')
for tweet in tweets:
    tweet_text = tweet.text
    if tweet_text.startswith('Sol'):
        mars_weather = tweet_text
        break
    else:
        continue

### Mars Hemispheres

In [13]:
hem_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hem_url)

In [14]:
urls = [(a.text, a['href']) for a in browser
         .find_by_css('div[class="description"] a')]

In [15]:
#defining a list to include all of the hemispheres
hemisphere_image_urls = []

In [16]:
for title,url in urls:
    product_dict = {}
    product_dict['title'] = title
    browser.visit(url)
    img_url = browser.find_by_css('img[class="wide-image"]')['src']
    product_dict['img_url'] = img_url
    hemisphere_image_urls.append(product_dict)

### Mars Facts

In [17]:
fact_url = "http://space-facts.com/mars/"

In [18]:
fact_table = pd.read_html(fact_url)
mars_fact_table = fact_table[0]
mars_fact_table 

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [19]:
mars_fact_table_html = mars_fact_table.to_html(header=False, index=False)

In [20]:
mars_fact_table_html = mars_fact_table_html.replace('\n', '')

In [25]:
browser.quit()